# Subdataset with random sampling

In [ ]:
from os import makedirs
from os.path import join

import numpy as np
import pandas as pd


from common import ALIASES, _filter_mset, read_toml

In [ ]:
config = read_toml('config.toml')
metachest_dir = config['metachest_dir']
df = pd.read_csv(join(metachest_dir, 'metachest.csv'))

mclasses = {'mtrn': config['mtrn'], 'mval': config['mval'], 'mtst': config['mtst']}
mset_dfs = [_filter_mset(mset, mclasses, df) for mset in mclasses.keys()]

distro_dir = join(metachest_dir, 'distro')
makedirs(distro_dir, exist_ok=True)

In [ ]:

def random_01(n, ones_pct=0.75):
    ones_n = int(n * ones_pct)
    nums = np.zeros(n, dtype=int)
    nums[:ones_n] = 1
    np.random.shuffle(nums)
    return nums

ptc = 0.75
ptc_str = f'{ptc:.2f}'[2:]

rand_mask = random_01(df.shape[0], ptc)
np.random.seed(0)
distro_df = pd.DataFrame({
    'mtrn': [1] * df.shape[0],
    'mval': rand_mask,
    'mtst': rand_mask
}).astype(int)

path = join(distro_dir, f'sub_random_{ptc_str}.csv')
distro_df.to_csv(path, index=False)
print(path)


### No-finding

In [ ]:
nf_df = pd.read_csv(join(metachest_dir, 'metachest_nf.csv'))

rand_mask = random_01(nf_df.shape[0], ptc)
nf_distro = ( (nf_df.mset == 0) |
             ((nf_df.mset == 1) & rand_mask) |
             ((nf_df.mset == 2) & rand_mask) )
nf_distro = nf_distro.rename('mask').to_frame().astype(int)
path = join(distro_dir, f'sub_random_{ptc_str}_nf.csv')
nf_distro.to_csv(path, index=False)
print(path)